# Walking-Forward VAR Model with all Variables - Weekly Data

In [84]:
#import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') 

#import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from pmdarima.model_selection import train_test_split

In [85]:
dvar1 = pd.read_csv("../Data/clean_data/weekly_data.csv")
dvar1

,datetime,suction_tension (20cm),suction_tension (40cm),suction_tension (70cm),water_temperature,barometric_pressure_qfe,dew_point,water_level,inflow_linth (m3/s),outflow_limmat (m3/s),precipitation (mm),Evapotranspiration (mm),change_rate
0,2012-01-08,1.864286,1.509524,1.657143,5.718254,972.342540,1.582381,405.925937,42.463125,130.712589,50.2,0.59,NaN
1,2012-01-15,3.045833,2.662500,2.254167,5.230734,981.800595,-0.039742,405.856155,41.259768,97.885887,0.0,0.72,-0.000172
2,2012-01-22,3.732738,2.593155,2.293155,4.960377,975.973750,-0.223948,405.811720,31.684387,85.611571,40.1,0.89,-0.000109
3,2012-01-29,1.999405,1.724405,1.831548,5.042589,973.659752,1.402569,405.828179,38.195262,101.169554,13.7,0.19,0.000041
4,2012-02-05,3.800000,2.969048,2.390476,4.035357,975.891310,-9.414226,405.776389,38.501393,81.974750,1.0,0.70,-0.000128
...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,2019-11-03,3.469347,2.576207,1.121059,14.486458,964.152153,8.988046,405.981652,49.719399,98.548613,35.8,0.60,0.000031
409,2019-11-10,2.369476,1.514638,0.815091,13.225020,957.176825,4.898849,406.003480,53.301202,99.244786,20.6,1.35,0.000054
410,2019-11-17,3.027560,2.366860,1.946473,11.813165,957.502272,1.784901,405.973247,49.496952,92.764923,5.4,1.63,-0.000074
411,2019-11-24,3.563155,2.414583,3.232589,10.580665,960.705625,2.868562,405.958633,46.694750,78.699708,0.0,1.14,-0.000036


In [86]:
dvar1 = dvar1.dropna()

In [87]:
# convert DateTime column into datetime format
dvar1["datetime"] = pd.to_datetime(dvar1['datetime'])
# set DateTime as index
dvar1.set_index("datetime", inplace=True)
dvar1.info()
dvar1

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 412 entries, 2012-01-15 to 2019-12-01
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   suction_tension (20cm)   412 non-null    float64
 1   suction_tension (40cm)   412 non-null    float64
 2   suction_tension (70cm)   412 non-null    float64
 3   water_temperature        412 non-null    float64
 4   barometric_pressure_qfe  412 non-null    float64
 5   dew_point                412 non-null    float64
 6   water_level              412 non-null    float64
 7   inflow_linth (m3/s)      412 non-null    float64
 8   outflow_limmat (m3/s)    412 non-null    float64
 9   precipitation (mm)       412 non-null    float64
 10  Evapotranspiration (mm)  412 non-null    float64
 11  change_rate              412 non-null    float64
dtypes: float64(12)
memory usage: 41.8 KB


,suction_tension (20cm),suction_tension (40cm),suction_tension (70cm),water_temperature,barometric_pressure_qfe,dew_point,water_level,inflow_linth (m3/s),outflow_limmat (m3/s),precipitation (mm),Evapotranspiration (mm),change_rate
datetime,,,,,,,,,,,,
2012-01-15,3.045833,2.662500,2.254167,5.230734,981.800595,-0.039742,405.856155,41.259768,97.885887,0.0,0.72,-0.000172
2012-01-22,3.732738,2.593155,2.293155,4.960377,975.973750,-0.223948,405.811720,31.684387,85.611571,40.1,0.89,-0.000109
2012-01-29,1.999405,1.724405,1.831548,5.042589,973.659752,1.402569,405.828179,38.195262,101.169554,13.7,0.19,0.000041
2012-02-05,3.800000,2.969048,2.390476,4.035357,975.891310,-9.414226,405.776389,38.501393,81.974750,1.0,0.70,-0.000128
2012-02-12,27.794643,4.252679,2.889286,3.024345,980.527917,-12.032123,405.751361,40.779726,78.217399,0.0,0.80,-0.000062
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-03,3.469347,2.576207,1.121059,14.486458,964.152153,8.988046,405.981652,49.719399,98.548613,35.8,0.60,0.000031
2019-11-10,2.369476,1.514638,0.815091,13.225020,957.176825,4.898849,406.003480,53.301202,99.244786,20.6,1.35,0.000054
2019-11-17,3.027560,2.366860,1.946473,11.813165,957.502272,1.784901,405.973247,49.496952,92.764923,5.4,1.63,-0.000074


In [88]:
from statsmodels.tsa.stattools import adfuller
def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.") 

In [89]:
for name, column in dvar1.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "suction_tension (20cm)" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -5.7781
 No. Lags Chosen       = 4
 Critical value 1%     = -3.447
 Critical value 5%     = -2.869
 Critical value 10%    = -2.571
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "suction_tension (40cm)" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.4793
 No. Lags Chosen       = 6
 Critical value 1%     = -3.447
 Critical value 5%     = -2.869
 Critical value 10%    = -2.571
 => P-Value = 0.0002. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "suction_tension (70cm)" 
    -----------------------------------------------
 Null Hypothesis: Data has 

As we can see all variables are stationary. However, when we controlled the monthly data, some variables turned out not to be stationary. Therefore we are going to difference for security.

# Split the data into train, validation and test set.

In [93]:
df = dvar1.drop(['water_temperature', 'suction_tension (20cm)', 'suction_tension (40cm)', 'suction_tension (70cm)', 'barometric_pressure_qfe', 'dew_point', 'Evapotranspiration (mm)', 'change_rate'], axis=1)
df

,water_level,inflow_linth (m3/s),outflow_limmat (m3/s),precipitation (mm)
datetime,,,,
2012-01-15,405.856155,41.259768,97.885887,0.0
2012-01-22,405.811720,31.684387,85.611571,40.1
2012-01-29,405.828179,38.195262,101.169554,13.7
2012-02-05,405.776389,38.501393,81.974750,1.0
2012-02-12,405.751361,40.779726,78.217399,0.0
...,...,...,...,...
2019-11-03,405.981652,49.719399,98.548613,35.8
2019-11-10,406.003480,53.301202,99.244786,20.6
2019-11-17,405.973247,49.496952,92.764923,5.4


In [94]:
train_val, test_val = train_test_split(df, train_size=314)

In [95]:
train_all, test_pred = train_test_split(df, train_size=364)

In [107]:
var_model = VARMAX(train_val, order=(4,0),enforce_stationarity= True)
fitted_model = var_model.fit(disp=False)
print(fitted_model.summary())

                                                             Statespace Model Results                                                            
Dep. Variable:     ['water_level', 'inflow_linth (m3/s)', 'outflow_limmat (m3/s)', 'precipitation (mm)']   No. Observations:                  314
Model:                                                                                            VAR(4)   Log Likelihood               -3177.665
                                                                                             + intercept   AIC                           6511.330
Date:                                                                                   Fri, 02 Dec 2022   BIC                           6803.783
Time:                                                                                           19:48:18   HQIC                          6628.189
Sample:                                                                                       01-15-2012                    

In [105]:
def walk_forward_validation_VAR(train, test, order):
    window=1
    forecast_series = pd.DataFrame(index=test.index, columns=[x for x in range(window)])
    training_data = train.copy()
    for i in test.index:
        model = VARMAX(training_data, order=order).fit()
        forecast_series.loc[i] = model.forecast(window)[0]
        training_data.loc[i] = test.loc[i]
        columns = [f'yt+{x}' for x in forecast_series.columns]
    forecast_series.columns = columns
    return forecast_series.astype(float), model

In [106]:
twk, tawk = walk_forward_validation_VAR(train_val[['water_level', 'outflow_limmat (m3/s)']], test_val[['water_level', 'outflow_limmat (m3/s)']], (3,1))

KeyError: 0